In [1]:
import pandas as pd
import src.scrape_insta
import src.clean as clean
%autoreload
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
%matplotlib inline
import pickle


In [3]:
qualdf = pd.read_csv('soft_content.txt', sep="|")

In [5]:
urls = list(qualdf.url.unique())
pickle_in2 = open("quant_data.pkl","rb")
quant_df = pickle.load(pickle_in2)

In [7]:
quant_df.head();

In [ ]:
#postdf2 = clean.clean_fresh_instagram_post_data(postdf)
postdf2_url_index = postdf2.set_index('url', drop=False)
qualdf_url_index = qualdf.set_index('url')

combined = pd.concat([postdf2_url_index, qualdf_url_index], axis=1)

In [91]:
def clean_combined(combined):
    combined = combined.drop(labels=['date_num', 'day','date','url','taken_at','num_posts','year','people_tagged','number_of_comments','hashtags','comments','commenters','caption','num_people'], axis=1)
    combined['male'] = combined['is_male'] == 1
    combined['female'] = combined['is_male'] == 0
    combined['mixed_gender'] = combined['is_male'] == 2
    combined['bikini'] = combined['bikini/apparel'] == 'b'
    combined['apparel'] = combined['bikini/apparel'] == 'a'
    combined['biki+apparel'] = combined['bikini/apparel'] == 'c'
    combined['no_product'] = combined['bikini/apparel'] == 'd'
    combined=combined.drop(['bikini/apparel','is_male'], axis=1)
    return combined

In [94]:
combined2=clean_combined(combined)

In [320]:
combined.shape

(105, 40)

In [226]:
y_comb = combined2['number_of_likes']
X_comb = combined2.drop(['number_of_likes'], axis=1)

X_comb_train, X_comb_test, y_comb_train, y_comb_test = train_test_split(X_comb.as_matrix(), y_comb.as_matrix(), test_size= .15)

In [227]:
gbr = GradientBoostingRegressor(learning_rate=0.1, loss='ls', n_estimators=100, random_state=1)

gbr.fit(X_comb_train,y_comb_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, presort='auto', random_state=1,
             subsample=1.0, verbose=0, warm_start=False)

In [228]:
y_hat_comb = gbr.predict(X_comb_test)
mse_comb = mean_squared_error(y_comb_test,y_hat_comb)
rse_comb = np.sqrt(mse_comb)
print("WITH COMBINED DATA")
print('mse = {}'.format(round(mse_comb,2)))
print('rse = {}'.format(round(rse_comb,2)))

WITH COMBINED DATA
mse = 6133.64
rse = 78.32


In [224]:
feats2 = gbr.feature_importances_

In [217]:
feats = gbr.feature_importances_

In [218]:
feats

array([0.09817831, 0.21603   , 0.11826235, 0.02365636, 0.00401511,
       0.04028335, 0.02482832, 0.03720151, 0.01378616, 0.0147315 ,
       0.02268171, 0.00207635, 0.02618405, 0.00210607, 0.00043331,
       0.01474979, 0.01652931, 0.05996796, 0.03396877, 0.03726308,
       0.02046913, 0.01677129, 0.00022131, 0.021929  , 0.0388441 ,
       0.00806883, 0.03688874, 0.01474272, 0.01357673, 0.02155476])

In [225]:
for name, imp, imp2 in zip(list(X_comb.columns),feats, feats2):
    print ("{} : {} | {}".format(name, round(imp,4),round(imp2,4)))

num_people_tagged : 0.0982 | 0.1124
month : 0.216 | 0.2205
hour : 0.1183 | 0.1199
DOW_0 : 0.0237 | 0.0013
DOW_1 : 0.004 | 0.0327
DOW_2 : 0.0403 | 0.0522
DOW_3 : 0.0248 | 0.0345
DOW_4 : 0.0372 | 0.0296
DOW_5 : 0.0138 | 0.0339
DOW_6 : 0.0147 | 0.0322
faces_visible : 0.0227 | 0.0305
sale : 0.0021 | 0.0
edited : 0.0262 | 0.0173
butt_pic : 0.0021 | 0.0042
ocean : 0.0004 | 0.0094
skate : 0.0147 | 0.0343
drift_content : 0.0165 | 0.0216
product_shot : 0.06 | 0.0125
lifestyle : 0.034 | 0.0239
in_shop : 0.0373 | 0.0313
pro\model : 0.0205 | 0.0207
in_nature : 0.0168 | 0.0088
surf : 0.0002 | 0.0048
male : 0.0219 | 0.0271
female : 0.0388 | 0.0103
mixed_gender : 0.0081 | 0.0259
bikini : 0.0369 | 0.0141
apparel : 0.0147 | 0.021
biki+apparel : 0.0136 | 0.0086
no_product : 0.0216 | 0.0046


# Just with quantitative data

In [321]:
postdf2.shape

(105, 24)

In [158]:
y_quan = postdf2['number_of_likes']
X_quan = postdf2[['DOW_0','DOW_1','DOW_2','DOW_3','DOW_4','DOW_5','DOW_6','hour','month','num_people_tagged']]

In [175]:
X_quan_train, X_quan_test, y_quan_train, y_quan_test = train_test_split(X_quan.as_matrix(), y_quan.as_matrix(), test_size= .15)

gbr= GradientBoostingRegressor(learning_rate=0.1, loss='ls', n_estimators=100)
gbr.fit(X_quan_train,y_quan_train)

y_hat_quan = gbr.predict(X_quan_test)
mse = mean_squared_error(y_quan_test, y_hat_quan)
rse = np.sqrt(mse)
print("WITH QUANTITATIVE DATA")
print('mse = {}'.format(round(mse,2)))
print('rse = {}'.format(round(rse,2)))

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, presort='auto', random_state=1,
             subsample=1.0, verbose=0, warm_start=False)

In [449]:
rest_insta = pd.read_csv('data/data.csv', index_col=0)

In [450]:
rest_insta2 = clean.clean_instagram_post_data(rest_insta)

In [249]:
rest_insta2.iloc[-1,:].year

2015

In [250]:
len(rest_insta[rest_insta['year']>=2017])

256

In [ ]:
hashtag_corpus = list(postdf2.hashtags)

hashtag_corpus2 = take_out_hash(hashtag_corpus)

hashtag_corpus3=[' '.join(hashtag_corpus2[i]) for i in range(len(hashtag_corpus2))]

tfidf_hash = TfidfVectorizer(input='content')

tfidf_hash.fit(hashtag_corpus3)
hash_mat =tfidf_hash.transform(hashtag_corpus3)
tfidf_hash_mat = pd.SparseDataFrame(hash_mat, index = postdf2.index).to_dense().fillna(0)

tfidf_hash_mat.shape

def take_out_sign(hashtag_corpus, sign='\#'):
    new_corp=[]
    for tags in hashtag_corpus:
        just_words = []
        for tag in tags:
            just_words.append(re.sub(sign,'',tag))
        new_corp.append(just_words)
    return new_corp 

In [266]:
def take_out_sign2(caption_corpus, sign='\#'):
    new_corp=[]
    for string in caption_corpus:
        string=re.sub(sign,'',string)
        new_corp.append(string)
    return new_corp 

In [292]:
caption_corpus  ;

In [272]:
caption_corpus = list(postdf2.caption)

In [297]:
caption_corpus2 = take_out_sign2(caption_corpus)
caption_corpus2 = take_out_sign2(caption_corpus2, sign= '\@')

In [314]:
tfidf = TfidfVectorizer(input='content', stop_words='english')

In [315]:
tfidf.fit(caption_corpus2)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words='english', strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [316]:
cap_mat =tfidf.transform(caption_corpus2)

tfidf_mat = pd.SparseDataFrame(cap_mat, index = postdf2.index).to_dense().fillna(0)

In [318]:
tfidf_mat.shape

(105, 818)

In [326]:
combined3 =pd.concat([combined2, tfidf_mat2], axis=1)

In [325]:
tfidf_mat2 = tfidf_mat.set_index(combined2.index)

In [431]:
y = combined4['number_of_likes']
X = combined4.drop(['number_of_likes'], axis=1)

In [432]:
X_train, X_test, y_train, y_test = train_test_split(X.as_matrix(), y.as_matrix(), test_size= .15)

GradientBoostingRegressor(learning_rate=0.001, loss='ls', n_estimators=10)
gbr.fit(X_train,y_train)

y_hat = gbr.predict(X_test)
mse = mean_squared_error(y_test, y_hat)
rse = np.sqrt(mse)
print("WITH QUANTITATIVE DATA")
print('mse = {}'.format(round(mse,2)))
print('rse = {}'.format(round(rse,2)))

WITH QUANTITATIVE DATA
mse = 2392.86
rse = 48.92


In [351]:
tfidf_hash_mat2 = tfidf_hash_mat.set_index(combined2.index)

In [353]:
combined4 = pd.concat([combined2, tfidf_hash_mat2], axis=1)
combined5 = pd.concat([combined3, tfidf_hash_mat2], axis=1)

In [355]:
combined5.shape

(105, 1073)

In [433]:
rest_insta2.head()

,caption,commenters,comments,hashtags,number_of_comments,number_of_likes,taken_at,url,date,people_tagged,...,day,hour,date_num,num_posts,DOW_1,DOW_2,DOW_3,DOW_4,DOW_5,DOW_6
0,SHOP LOCAL 🌈 @pioneersboardshop @cinnamonrai...,"[nolongersauced, thedriftcollective]","[I <3 bethel skatepark, .\n.\n.\n.\n.\n#summer...",[],2,300,1515166569,http://www.instagram.com/p/BdkrH7uHkpr/?taken-...,2018-01-05 15:36:09,"[nolongersauced, pioneersboardshop, cinnamonra...",...,5,15,20180105,1,0,0,0,1,0,0
1,Snow day 🍻 Shop will reopen tomorrow at noon! ...,"[wyndhamvacationrentals, demetriusortakales, h...",[We'd love to feature your photo! Can we shar...,"[#summersunselection, #createexploretakeover, ...",3,468,1515078603,http://www.instagram.com/p/BdiDVyynAdH/?taken-...,2018-01-04 15:10:03,"[rhlockhart, thedriftcollective, rhlockhart]",...,4,15,20180104,1,0,0,1,0,0,0
2,Shop open again tomorrow! 🛷🌴 Brave the cold + ...,"[johnnyoconnor, zynenwartel, thedriftcollectiv...","[I gotta come try this on, 👌👌, @johnnyoconnor ...","[#summersunselection, #urbanstyle, #modernstyl...",6,244,1514933720,http://www.instagram.com/p/Bddu_4Vn5pM/?taken-...,2018-01-02 22:55:20,[thedriftcollective],...,2,22,20180102,1,1,0,0,0,0,0
3,Merry Christmas Eve ⭐️ Come celebrate and fini...,"[micky_kimm, theprosperitygirl]","[😊😄, 😊😊✌...]","[#vintagechristmas, #merrychristmaseve, #summe...",2,258,1514130168,http://www.instagram.com/p/BdFyWJOnkVz/?taken-...,2017-12-24 15:42:48,"[elisabettalockhart, sea__soul, slimaarons, sl...",...,24,15,20171224,1,0,0,0,0,0,1
4,🔥🔥🔥 You deserve to dress one of a kind. Our di...,"[hdcostinyamoney, veilleuxkristen, westvillage...","[Love it🤗, @hdcostinyamoney @juliiivanegas fin...","[#shopsustainable, #summersunselection, #urban...",9,296,1514059594,http://www.instagram.com/p/BdDrvIOn_nk/?taken-...,2017-12-23 20:06:34,"[hdcostinyamoney, veilleuxkristen, thedriftcol...",...,23,20,20171223,1,0,0,0,0,1,0


In [453]:
all_quant_with_dup = pd.concat([rest_insta2, postdf2])

In [455]:
len(all_quant_with_dup)

500

In [461]:
all_quant = all_quant_with_dup.drop_duplicates(subset='caption')

In [520]:
pickle_out = open("quant_data.pkl","wb")
pickle.dump(all_quant, pickle_out)
pickle_out.close()

In [467]:
all2017on = all_quant[all_quant['year']>=2017]

In [468]:
len(all2017on)

270

In [472]:
ppl_tagged = list(all_quant.people_tagged)

In [477]:
ppl_ever_tagged = {}
for ppl in ppl_tagged:
    for per in ppl:
        if per not in ppl_ever_tagged:
            ppl_ever_tagged[per]=1
        else:    
            ppl_ever_tagged[per]+=1

In [486]:
ppl_ever_tagged;

In [481]:
import pickle


pickle_out = open("ppl_ever_tagged.pkl","wb")
pickle.dump(ppl_ever_tagged, pickle_out)
pickle_out.close()

In [483]:
pickle_in = open("ppl_ever_tagged.pkl","rb")
example_dict = pickle.load(pickle_in)

In [487]:
example_dict;

In [517]:
serious_reps=[]
mid_reps = []
lesser_reps = []
for key, val in ppl_ever_tagged.items():
    if val >= 10:
        serious_reps.append(key)
    if val >= 4 and val < 6:
        lesser_reps.append(key)
    if val >= 6 and val <10:
        mid_reps.append(key)

In [518]:
print(len(lesser_reps))
print(len(mid_reps))
print(len(serious_reps))

18
18
19


In [ ]:
y = combined4['number_of_likes']
X = combined4.drop(['number_of_likes'], axis=1)

In [635]:
gbr = GradientBoostingRegressor(learning_rate=0.01, loss='ls', min_samples_split=4, n_estimators=600, max_depth=2)
scores = cross_val_score(gbr, X, y,cv=40, scoring= 'neg_mean_absolute_error')
print(scores.mean())
print(scores)

-48.480377205046416
[ -38.15759618  -41.46070029   -7.06369215  -50.27301336  -62.2982722
  -91.69993866  -18.91128562  -44.17522856  -51.58734248  -16.2278911
  -75.66074756  -83.00390288  -45.78961711  -90.48344108  -48.3360088
  -27.46482184  -54.77182226  -43.95735829  -66.76219801  -36.21908022
  -27.44591783  -55.99077238  -12.21049814  -21.17192359  -66.35742076
  -57.76466755  -21.8554863   -22.1195746   -48.25592832  -15.11909521
  -50.94315015  -36.71826243  -64.5390411   -21.46128815  -83.65685558
  -65.09989994  -46.60912983  -82.65519532  -24.50847012 -120.42855226]


In [632]:
_=gbr.fit(X.as_matrix(),y.as_matrix())

In [633]:
for name, imp in zip(X.columns,gbr.feature_importances_):
    if imp > .015:
        print('{} : {}'.format(name, round(imp,3)))

num_people_tagged : 0.043
month : 0.02
hour : 0.032
in_shop : 0.042
pro\model : 0.042
no_product : 0.032
16 : 0.017
48 : 0.022
58 : 0.021
59 : 0.023
62 : 0.02
71 : 0.034
84 : 0.022
88 : 0.053
91 : 0.015
102 : 0.034
110 : 0.024
111 : 0.076
115 : 0.046
116 : 0.016
131 : 0.047
150 : 0.018
158 : 0.034
186 : 0.023
203 : 0.025
205 : 0.044
220 : 0.025
223 : 0.017


In [524]:
from sklearn.model_selection import cross_val_score


In [557]:
postdf2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105 entries, 0 to 104
Data columns (total 24 columns):
caption               105 non-null object
commenters            105 non-null object
comments              105 non-null object
hashtags              105 non-null object
number_of_comments    105 non-null int64
number_of_likes       105 non-null int64
people_tagged         105 non-null object
taken_at              105 non-null int64
url                   105 non-null object
date                  105 non-null datetime64[ns]
num_people_tagged     105 non-null int64
year                  105 non-null int64
month                 105 non-null int64
day                   105 non-null int64
hour                  105 non-null int64
date_num              105 non-null int64
num_posts             105 non-null int64
DOW_0                 105 non-null uint8
DOW_1                 105 non-null uint8
DOW_2                 105 non-null uint8
DOW_3                 105 non-null uint8
DOW_4              

In [9]:
to_drop=['number_of_likes','caption','commenters','comments','date', 'date_num','hashtags', 'num_posts', 'number_of_comments','people_tagged','taken_at', 'url']

In [ ]:
time lag of insta to shopsales